<a href="https://colab.research.google.com/github/crashidian/CSC510/blob/main/CSC_510_Critical_Thinking_No_4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from collections import deque

In [2]:
class WarehouseState:
    def __init__(self, grid, robot_pos, items_to_collect):
        self.grid = grid
        self.robot_pos = robot_pos
        self.items_to_collect = items_to_collect

    def is_goal(self):
        return len(self.items_to_collect) == 0 and self.grid[self.robot_pos[0]][self.robot_pos[1]] == 'P'

    def get_successors(self):
        successors = []
        x, y = self.robot_pos
        directions = [(0, 1), (0, -1), (1, 0), (-1, 0)]

        for dx, dy in directions:
            new_x, new_y = x + dx, y + dy
            if 0 <= new_x < len(self.grid) and 0 <= new_y < len(self.grid[0]):
                if self.grid[new_x][new_y] != '#':
                    new_items_to_collect = set(self.items_to_collect)
                    if self.grid[new_x][new_y] in new_items_to_collect:
                        new_items_to_collect.remove(self.grid[new_x][new_y])
                    successors.append(WarehouseState(self.grid, (new_x, new_y), new_items_to_collect))

        return successors

    def heuristic(self):
        if len(self.items_to_collect) == 0:
            return manhattan_distance(self.robot_pos, find_packaging_area(self.grid))
        else:
            return min(manhattan_distance(self.robot_pos, find_item(self.grid, item)) for item in self.items_to_collect)


In [3]:
def find_item(grid, item):
    for i in range(len(grid)):
        for j in range(len(grid[0])):
            if grid[i][j] == item:
                return i, j
    return None

In [4]:
def find_packaging_area(grid):
    for i in range(len(grid)):
        for j in range(len(grid[0])):
            if grid[i][j] == 'P':
                return i, j
    return None

In [5]:
def manhattan_distance(pos1, pos2):
    return abs(pos1[0] - pos2[0]) + abs(pos1[1] - pos2[1])

In [6]:
def best_first_search(initial_state):
    open_list = deque([(initial_state, [])])
    closed_set = set()

    while open_list:
        current_state, path = open_list.popleft()
        if current_state.is_goal():
            return path + [current_state]
        if tuple(current_state.robot_pos) not in closed_set:
            closed_set.add(tuple(current_state.robot_pos))
            successors = current_state.get_successors()
            for successor in successors:
                open_list.append((successor, path + [current_state]))
            open_list = deque(sorted(open_list, key=lambda x: x[0].heuristic()))

    return None

In [7]:
# Example usage
warehouse_grid = [
    ['#', '#', '#', '#', '#'],
    ['#', 'A', '.', 'B', '#'],
    ['#', '.', '#', '.', '#'],
    ['#', 'P', '.', 'C', '#'],
    ['#', '#', '#', '#', '#']
]

initial_pos = (1, 1)
items_to_collect = {'B', 'C'}
initial_state = WarehouseState(warehouse_grid, initial_pos, items_to_collect)

solution_path = best_first_search(initial_state)

In [8]:
if solution_path:
    print("Solution found:")
    for state in solution_path:
        print(state.robot_pos, state.items_to_collect)
else:
    print("No solution found.")

Solution found:
(1, 1) {'B', 'C'}
(1, 2) {'B', 'C'}
(1, 3) {'C'}
(2, 3) {'C'}
(3, 3) set()
(3, 2) set()
(3, 1) set()
